**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [18]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [19]:
# Import the data set
df = pd.read_csv('cleaned_data.csv')
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [20]:
# Define the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [21]:
x = df[['LIMIT_BAL', 'PAY_1']]
y = df['default payment next month']

In [22]:
""""from sklearn import preprocessing
x = preprocessing.StandardScaler().fit_transform(x)
y = np.asarray(y)
""""

SyntaxError: EOL while scanning string literal (<ipython-input-22-6dc4491c86af>, line 4)

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [23]:
# Create a train/test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 24)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(21331, 2) (21331,)
(5333, 2) (5333,)


______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [24]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver = 'liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [25]:
# Fit the logistic regression model on training data
lr.fit(x_train, y_train)

LogisticRegression(solver='liblinear')

In [26]:
# Make predictions using `.predict()`
y_predict = lr.predict(x_test)

In [27]:
# Find class probabilities using `.predict_proba()`
y_pred_prob = lr.predict_proba(x_test)
y_pred_prob

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [28]:
# Add column of 1s to features
dimension = np.hstack([np.ones((x_test.shape[0],1)), x_test])
dimension

array([[ 1.0e+00,  1.6e+05,  2.0e+00],
       [ 1.0e+00,  5.0e+04,  1.0e+00],
       [ 1.0e+00,  2.0e+05, -1.0e+00],
       ...,
       [ 1.0e+00,  5.0e+04, -1.0e+00],
       [ 1.0e+00,  2.3e+05,  1.0e+00],
       [ 1.0e+00,  1.0e+05,  2.0e+00]])

In [29]:
""""x_test = np.append(x_test, ones, axis = 1)
x_test.shape
""""

SyntaxError: EOL while scanning string literal (<ipython-input-29-3625d83da795>, line 3)

In [30]:
# Get coefficients and intercepts from trained model
coefficient = lr.coef_
intercept = lr.intercept_
print('Coefficients: ', coefficient)
print('Intercepts: ', intercept)

Coefficients:  [[-6.80876727e-06  8.27451187e-11]]
Intercepts:  [-6.57647457e-11]


In [33]:
coefficient_intercept = np.concatenate([intercept.reshape(1,1), coefficient], axis = 1)
coefficient_intercept

array([[-6.57647457e-11, -6.80876727e-06,  8.27451187e-11]])

In [35]:
# Manually calculate predicted probabilities
z = np.dot(coefficient_intercept, dimension.T)
z

array([[-1.08940276, -0.34043836, -1.36175345, ..., -0.34043836,
        -1.56601647, -0.68087673]])

In [36]:
probablity = sigmoid(z)
probablity

array([[0.25173076, 0.415703  , 0.20395547, ..., 0.415703  , 0.17278502,
        0.33606565]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [50]:
# Manually calculate predicted classes
manual_pred = z >= 0.5

array([[False, False, False, ..., False, False, False]])

In [39]:
# Compare to scikit-learn's predicted classes
print(y_predict.shape)
print(manual_pred.shape)

(5333,)
(1, 5333)


______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [40]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score, roc_curve
rocauc_score = roc_auc_score(y_test, y_pred_prob[:, 1])
rocauc_score

0.627207450280691

In [67]:
# Use manually calculated predicted probabilities to calculate ROC AUC
rocauc_score = roc_auc_score(y_test, z.reshape(z.shape[1]))
rocauc_score

0.627207450280691